In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_RAW = Path("../data/raw")
DATA_PROCESSED = Path("../data/processed")

df = pd.read_csv(DATA_RAW / "ecommerce_sales.csv")

df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Order ID       5000 non-null   int64  
 1   Order Date     5000 non-null   object 
 2   Customer Name  5000 non-null   object 
 3   Region         5000 non-null   object 
 4   City           5000 non-null   object 
 5   Category       5000 non-null   object 
 6   Sub-Category   5000 non-null   object 
 7   Product Name   5000 non-null   object 
 8   Quantity       5000 non-null   int64  
 9   Unit Price     5000 non-null   int64  
 10  Discount       5000 non-null   int64  
 11  Sales          5000 non-null   float64
 12  Profit         5000 non-null   float64
 13  Payment Mode   5000 non-null   object 
dtypes: float64(2), int64(4), object(8)
memory usage: 547.0+ KB


In [2]:
# make a copy to be safe
data = df.copy()

# standardize column names (lowercase, underscores)
data.columns = (
    data.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

data.columns


Index(['order_id', 'order_date', 'customer_name', 'region', 'city', 'category',
       'sub_category', 'product_name', 'quantity', 'unit_price', 'discount',
       'sales', 'profit', 'payment_mode'],
      dtype='object')

In [3]:
data.columns.tolist()



['order_id',
 'order_date',
 'customer_name',
 'region',
 'city',
 'category',
 'sub_category',
 'product_name',
 'quantity',
 'unit_price',
 'discount',
 'sales',
 'profit',
 'payment_mode']

In [4]:
# Choose candidate date columns – adjust names if your dataset uses different ones
candidate_date_cols = ["order_date", "ship_date"]

# Keep only the ones that actually exist in the DataFrame
date_cols = [col for col in candidate_date_cols if col in data.columns]

print("Date columns being parsed:", date_cols)

for col in date_cols:
    data[col] = pd.to_datetime(data[col], errors="coerce")

# Only select existing date columns here
if date_cols:
    display(data[date_cols].head())
    print(data[date_cols].isna().sum())
else:
    print("No date columns found in data – check your column names.")


Date columns being parsed: ['order_date']


,order_date
0,2024-10-19
1,2025-08-30
2,2023-11-04
3,2025-05-23
4,2025-01-19


order_date    0
dtype: int64


In [5]:
data["order_date"] = pd.to_datetime(data["order_date"], errors="coerce")
data["order_date"].head()
data["order_date"].isna().sum()


np.int64(0)

In [6]:
data.columns.tolist()


['order_id',
 'order_date',
 'customer_name',
 'region',
 'city',
 'category',
 'sub_category',
 'product_name',
 'quantity',
 'unit_price',
 'discount',
 'sales',
 'profit',
 'payment_mode']

In [7]:
# Candidate date columns (adjust if your dataset uses different names)
candidate_date_cols = ["order_date", "ship_date"]

# Keep only those that actually exist in the DataFrame
date_cols = [col for col in candidate_date_cols if col in data.columns]

print("Date columns being parsed:", date_cols)

for col in date_cols:
    data[col] = pd.to_datetime(data[col], errors="coerce")

if date_cols:
    display(data[date_cols].head())
    print("Missing values per date col:")
    print(data[date_cols].isna().sum())
else:
    print("No date columns found. Check your column names above.")



Date columns being parsed: ['order_date']


,order_date
0,2024-10-19
1,2025-08-30
2,2023-11-04
3,2025-05-23
4,2025-01-19


Missing values per date col:
order_date    0
dtype: int64


In [8]:
# Check missing values ratio per column
missing_ratio = data.isna().mean().sort_values(ascending=False)
missing_ratio.head(20)


order_id         0.0
order_date       0.0
customer_name    0.0
region           0.0
city             0.0
category         0.0
sub_category     0.0
product_name     0.0
quantity         0.0
unit_price       0.0
discount         0.0
sales            0.0
profit           0.0
payment_mode     0.0
dtype: float64

In [9]:
for col in ["sales", "quantity", "discount", "profit"]:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors="coerce")

data[["sales", "quantity", "discount", "profit"]].describe()


,sales,quantity,discount,profit
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,106733.204870,2.992600,10.051000,15941.746982
std,85108.208202,1.413133,7.084662,14897.684916
min,264.100000,1.000000,0.000000,19.120000
25%,39766.537500,2.000000,5.000000,4892.295000
50%,83080.325000,3.000000,10.000000,11108.525000
75%,156968.587500,4.000000,15.000000,22467.987500
max,398485.000000,5.000000,20.000000,89688.440000


In [10]:
# Drop rows with non-positive quantity or sales if they exist
if "quantity" in data.columns:
    before = len(data)
    data = data[data["quantity"] > 0]
    print("Dropped rows with quantity <= 0:", before - len(data))

if "sales" in data.columns:
    before = len(data)
    data = data[data["sales"] > 0]
    print("Dropped rows with sales <= 0:", before - len(data))


Dropped rows with quantity <= 0: 0
Dropped rows with sales <= 0: 0


In [11]:
# ORDERS TABLE
orders_cols = [
    "order_id",
    "order_date",
    "customer_id",
    "product_id",
    "segment",
    "country",
    "city",
    "region",
    "category",
    "sub_category",
    "sales",
    "quantity",
    "discount",
    "profit",
]

orders_cols = [c for c in orders_cols if c in data.columns]

orders = data[orders_cols].copy()
print("Orders shape:", orders.shape)
orders.head()


Orders shape: (5000, 10)


,order_id,order_date,city,region,category,sub_category,sales,quantity,discount,profit
0,10001,2024-10-19,Bangalore,South,Books,Non-Fiction,68958.6,2,5,10525.09
1,10002,2025-08-30,Delhi,North,Groceries,Rice,33732.0,1,20,6299.66
2,10003,2023-11-04,Patna,East,Kitchen,Juicer,207603.2,4,20,19850.27
3,10004,2025-05-23,Kolkata,East,Groceries,Oil,158610.0,5,15,36311.02
4,10005,2025-01-19,Pune,West,Clothing,Kids Wear,45033.3,1,10,9050.04


In [12]:
# Ensure we have a customer_id column
# If dataset doesn't have one, we create a synthetic ID from customer_name.

print("Columns in data:", data.columns.tolist())

if "customer_id" not in data.columns:
    if "customer_name" in data.columns:
        # Create a stable ID from customer_name
        data["customer_id"] = (
            data["customer_name"]
            .astype("category")
            .cat.codes
            .astype(int)
            + 1
        ).astype(str)

        data["customer_id"] = "C" + data["customer_id"]
        print("Created synthetic customer_id from customer_name.")
    else:
        raise ValueError(
            "No 'customer_id' or 'customer_name' column found. "
            "Check your dataset columns and adapt the code."
        )

# Quick check
data[["customer_id"]].head()



Columns in data: ['order_id', 'order_date', 'customer_name', 'region', 'city', 'category', 'sub_category', 'product_name', 'quantity', 'unit_price', 'discount', 'sales', 'profit', 'payment_mode']
Created synthetic customer_id from customer_name.


,customer_id
0,C2093
1,C210
2,C3419
3,C475
4,C266


In [13]:
# CUSTOMERS TABLE
customer_cols = [
    "customer_id",
    "customer_name",
    "segment",
    "country",
    "city",
    "region",
]

customer_cols = [c for c in customer_cols if c in data.columns]

customers = data[customer_cols].copy()

# Use customer_id for uniqueness if available
if "customer_id" in customers.columns:
    customers = customers.drop_duplicates(subset=["customer_id"])
else:
    customers = customers.drop_duplicates()

customers = customers.reset_index(drop=True)

print("Customers shape:", customers.shape)
customers.head()


Customers shape: (4844, 4)


,customer_id,customer_name,city,region
0,C2093,Kashvi Varty,Bangalore,South
1,C210,Advik Desai,Delhi,North
2,C3419,Rhea Kalla,Patna,East
3,C475,Anika Sen,Kolkata,East
4,C266,Akarsh Kaul,Pune,West


In [14]:
# ORDERS TABLE
orders_cols = [
    "order_id",
    "order_date",
    "customer_id",   # now guaranteed to exist from previous step
    "product_id",
    "segment",
    "country",
    "city",
    "region",
    "category",
    "sub_category",
    "sales",
    "quantity",
    "discount",
    "profit",
]

orders_cols = [c for c in orders_cols if c in data.columns]

orders = data[orders_cols].copy()
print("Orders shape:", orders.shape)
orders.head()


Orders shape: (5000, 11)


,order_id,order_date,customer_id,city,region,category,sub_category,sales,quantity,discount,profit
0,10001,2024-10-19,C2093,Bangalore,South,Books,Non-Fiction,68958.6,2,5,10525.09
1,10002,2025-08-30,C210,Delhi,North,Groceries,Rice,33732.0,1,20,6299.66
2,10003,2023-11-04,C3419,Patna,East,Kitchen,Juicer,207603.2,4,20,19850.27
3,10004,2025-05-23,C475,Kolkata,East,Groceries,Oil,158610.0,5,15,36311.02
4,10005,2025-01-19,C266,Pune,West,Clothing,Kids Wear,45033.3,1,10,9050.04


In [15]:
# Ensure we have a product_id column
# If dataset doesn't have one, create a synthetic ID from product_name.

print("Columns in data:", data.columns.tolist())

if "product_id" not in data.columns:
    if "product_name" in data.columns:
        data["product_id"] = (
            data["product_name"]
            .astype("category")
            .cat.codes
            .astype(int)
            + 1
        ).astype(str)

        data["product_id"] = "P" + data["product_id"]
        print("Created synthetic product_id from product_name.")
    else:
        raise ValueError(
            "No 'product_id' or 'product_name' column found. "
            "Check your dataset columns and adapt the code."
        )

data[["product_id"]].head()



Columns in data: ['order_id', 'order_date', 'customer_name', 'region', 'city', 'category', 'sub_category', 'product_name', 'quantity', 'unit_price', 'discount', 'sales', 'profit', 'payment_mode', 'customer_id']
Created synthetic product_id from product_name.


,product_id
0,P2259
1,P2721
2,P1590
3,P2328
4,P1677


In [16]:
# PRODUCTS TABLE
product_cols = [
    "product_id",
    "product_name",
    "category",
    "sub_category",
]

product_cols = [c for c in product_cols if c in data.columns]

products = data[product_cols].copy()

# Use product_id if available; otherwise just drop exact duplicates
if "product_id" in products.columns:
    products = products.drop_duplicates(subset=["product_id"])
else:
    products = products.drop_duplicates()

products = products.reset_index(drop=True)

print("Products shape:", products.shape)
products.head()


Products shape: (3835, 4)


,product_id,product_name,category,sub_category
0,P2259,Non-Fiction Ipsum,Books,Non-Fiction
1,P2721,Rice Nemo,Groceries,Rice
2,P1590,Juicer Odio,Kitchen,Juicer
3,P2328,Oil Doloribus,Groceries,Oil
4,P1677,Kids Wear Quo,Clothing,Kids Wear


In [ ]:
import pandas as pd
from pathlib import Path

DATA_PROCESSED = Path("../data/processed")

orders = pd.read_csv(DATA_PROCESSED / "orders_clean.csv")
print(orders.columns.tolist())


In [27]:
from pathlib import Path

DATA_PROCESSED = Path("../data/processed")

for name in ["orders_clean.csv", "customers_clean.csv", "products_clean.csv"]:
    f = DATA_PROCESSED / name
    if f.exists():
        print("Removing existing file:", f)
        f.unlink()
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

orders.to_csv(DATA_PROCESSED / "orders_clean.csv", index=False)
customers.to_csv(DATA_PROCESSED / "customers_clean.csv", index=False)
products.to_csv(DATA_PROCESSED / "products_clean.csv", index=False)

print("Saved:")
print(DATA_PROCESSED / "orders_clean.csv")
print(DATA_PROCESSED / "customers_clean.csv")
print(DATA_PROCESSED / "products_clean.csv")


Removing existing file: ..\data\processed\orders_clean.csv
Removing existing file: ..\data\processed\products_clean.csv
Saved:
..\data\processed\orders_clean.csv
..\data\processed\customers_clean.csv
..\data\processed\products_clean.csv


In [22]:
# Ensure we have a product_id column
print("Columns in data:", data.columns.tolist())

if "product_id" not in data.columns:
    if "product_name" in data.columns:
        data["product_id"] = (
            data["product_name"]
            .astype("category")
            .cat.codes
            .astype(int)
            + 1
        ).astype(str)

        data["product_id"] = "P" + data["product_id"]
        print("Created synthetic product_id from product_name.")
    else:
        raise ValueError(
            "No 'product_id' or 'product_name' column found. "
            "Check your dataset columns and adapt the code."
        )

data[["product_id", "product_name"]].head()


Columns in data: ['order_id', 'order_date', 'customer_name', 'region', 'city', 'category', 'sub_category', 'product_name', 'quantity', 'unit_price', 'discount', 'sales', 'profit', 'payment_mode', 'customer_id', 'product_id']


,product_id,product_name
0,P2259,Non-Fiction Ipsum
1,P2721,Rice Nemo
2,P1590,Juicer Odio
3,P2328,Oil Doloribus
4,P1677,Kids Wear Quo


In [23]:
# ORDERS TABLE
orders_cols = [
    "order_id",
    "order_date",
    "customer_id",
    "product_id",   # <-- THIS MUST BE HERE
    "segment",
    "country",
    "city",
    "region",
    "category",
    "sub_category",
    "sales",
    "quantity",
    "discount",
    "profit",
]

orders_cols = [c for c in orders_cols if c in data.columns]

orders = data[orders_cols].copy()
print("Orders shape:", orders.shape)
orders.head()


Orders shape: (5000, 12)


,order_id,order_date,customer_id,product_id,city,region,category,sub_category,sales,quantity,discount,profit
0,10001,2024-10-19,C2093,P2259,Bangalore,South,Books,Non-Fiction,68958.6,2,5,10525.09
1,10002,2025-08-30,C210,P2721,Delhi,North,Groceries,Rice,33732.0,1,20,6299.66
2,10003,2023-11-04,C3419,P1590,Patna,East,Kitchen,Juicer,207603.2,4,20,19850.27
3,10004,2025-05-23,C475,P2328,Kolkata,East,Groceries,Oil,158610.0,5,15,36311.02
4,10005,2025-01-19,C266,P1677,Pune,West,Clothing,Kids Wear,45033.3,1,10,9050.04


In [24]:
# PRODUCTS TABLE
product_cols = [
    "product_id",
    "product_name",
    "category",
    "sub_category",
]

product_cols = [c for c in product_cols if c in data.columns]

products = data[product_cols].copy()

if "product_id" in products.columns:
    products = products.drop_duplicates(subset=["product_id"])
else:
    products = products.drop_duplicates()

products = products.reset_index(drop=True)

print("Products shape:", products.shape)
products.head()


Products shape: (3835, 4)


,product_id,product_name,category,sub_category
0,P2259,Non-Fiction Ipsum,Books,Non-Fiction
1,P2721,Rice Nemo,Groceries,Rice
2,P1590,Juicer Odio,Kitchen,Juicer
3,P2328,Oil Doloribus,Groceries,Oil
4,P1677,Kids Wear Quo,Clothing,Kids Wear


In [28]:
orders = pd.read_csv("../data/processed/orders_clean.csv")
print(orders.columns.tolist())


['order_id', 'order_date', 'customer_id', 'product_id', 'city', 'region', 'category', 'sub_category', 'sales', 'quantity', 'discount', 'profit']
